In [1]:
import cv2
import numpy as np
import torch
from torch.autograd import Variable
from torchvision import models
import matplotlib.pyplot as plt
import os
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable

c:\Users\Reinaldo yang\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##**Example1~5**##

In [4]:
def preprocess_image(cv2im, resize_im=True):

    # Resize image
    if resize_im:
        cv2im = cv2.resize(cv2im, (224, 224))
    im_as_arr = np.float32(cv2im)
    im_as_arr = np.ascontiguousarray(im_as_arr[..., ::-1])
    im_as_arr = im_as_arr.transpose(2, 0, 1)  # Convert array to D,W,H
    # Normalize the channels
    for channel, _ in enumerate(im_as_arr):
        im_as_arr[channel] /= 255
    # Convert to float tensor
    im_as_ten = torch.from_numpy(im_as_arr).float()
    # Add one more channel to the beginning. Tensor shape = 1,3,224,224
    im_as_ten.unsqueeze_(0)
    # Convert to Pytorch variable
    im_as_var = Variable(im_as_ten, requires_grad=True)
    return im_as_var

In [5]:
class FeatureVisualization():
    def __init__(self,img_path,selected_layer,models=models.vgg16(pretrained=True)):
        self.img_path=img_path
        self.selected_layer=selected_layer
        # if models==None:
        #   models = models.vgg16(pretrained=True)
        # else:
        #   models = models
        # Load pretrained model
        
        self.pretrained_model = models.features.cuda()
        self.pretrained_model.eval()
        self.pretrained_model2 = models.cuda()
        #self.pretrained_model2 = models.resnet18(pretrained=True)
        self.pretrained_model2.eval()
    def process_image(self):
        img=cv2.imread(self.img_path)
        img=preprocess_image(img)
        return img

    def get_feature(self):
        # Image  preprocessing
        input=self.process_image()
        #print("input.shape:{}".format(input.shape))
        x=input.cuda()
        for index,layer in enumerate(self.pretrained_model):
            x=layer(x)
            #print("x:{}".format(x.shape))
            if (index == self.selected_layer):
                return x

    def get_single_feature(self):
        # Get the feature map

        features=self.get_feature()
        print(features.shape)
        feature=features[:,0,:,:]
        feature=feature.view(feature.shape[1],feature.shape[2])

        #print("feature")
        #print(feature.shape)
        return feature

    def get_multi_feature(self):
        # Get the feature map
        features=self.get_feature()
        #print(features.shape)
        result_path = './feat_first' + str(self.selected_layer)

        if not os.path.exists(result_path):
            os.makedirs(result_path)
        print("On layer:{}, We can get the {} feature maps".format(self.selected_layer,features.shape[1]))    
        #print(features.shape[1])
        for i in range(features.shape[1]):
            feature=features[:,i,:,:]
            feature=feature.view(feature.shape[1],feature.shape[2])
            feature = feature.data.cpu().numpy()
            feature = 1.0 / (1 + np.exp(-1 * feature))
            feature = np.round(feature * 255)
            save_name = result_path + '/' + str(i) + '.jpg'
            cv2.imwrite(save_name, feature)
    def get_multi_feature1(self):
        # Get the feature map
        features=self.get_feature()
        #print(features.shape)
        result_path = './feat_second' + str(self.selected_layer)

        if not os.path.exists(result_path):
            os.makedirs(result_path)
        print("On layer:{}, We can get the {} feature maps".format(self.selected_layer,features.shape[1]))    
        #print(features.shape[1])
        for i in range(features.shape[1]):
            feature=features[:,i,:,:]
            feature=feature.view(feature.shape[1],feature.shape[2])
            feature = feature.data.cpu().numpy()
            feature = 1.0 / (1 + np.exp(-1 * feature))
            feature = np.round(feature * 255)
            save_name = result_path + '/' + str(i) + '.jpg'
            cv2.imwrite(save_name, feature)

    def save_feature_to_img(self,epoch=None):
        #to numpy
        feature=self.get_single_feature()
        self.get_multi_feature()
        feature=feature.data.cpu().numpy()

        #use sigmod to [0,1]
        # print(feature[0])
        feature= 1.0/(1+np.exp(-1*feature))

        # to [0,255]
        feature=np.round(feature*255)
        #print(self.selected_layer)
        if epoch==None:
          save_name = './feat_first' + str(self.selected_layer) + '.jpg'
          cv2.imwrite(save_name, feature)
        else:
          save_path = './feature_per_epoch/'
          if not os.path.exists(save_path):
            os.makedirs(save_path)
          save_name = save_path+ 'feat_first' + str(self.selected_layer) +'_'+str(epoch)+ '.jpg'
          
          cv2.imwrite(save_name, feature)
    def save_feature_to_img1(self):
        #to numpy
        feature=self.get_single_feature()
        self.get_multi_feature1()
        feature=feature.data.cpu().numpy()

        #use sigmod to [0,1]
        # print(feature[0])
        feature= 1.0/(1+np.exp(-1*feature))

        # to [0,255]
        feature=np.round(feature*255)
        #print(self.selected_layer)
        save_name = './feat_second' + str(self.selected_layer) + '.jpg'
        cv2.imwrite(save_name, feature)
    def plot_probablity(self,outputs):

        outputs = outputs.data.cpu().numpy()
        outputs = np.ndarray.tolist(outputs)
        print(outputs[0])
        x = range(0, 1000)
        plt.bar(x, outputs[0])
        plt.xlabel("Class")
        plt.ylabel("Probablity")
        plt.title("Image classifier")
        plt.show()


    def predict(self):
        input=self.process_image().cuda()
        outputs = self.pretrained_model2(input)
        
        s = torch.nn.Softmax(dim=1)
        result = s(outputs)
        # self.plot_probablity(result)
        
        prob, predicted = result.sort(1,descending=True)
        prob = prob.data.cpu().numpy()

        predicted = predicted.data.cpu().numpy()
        
        print("Probablity TOP-3:\n")
        print("")
        for i in range(3):
            
            print("TOP_"+str(i+1))
            print("Probablity:{}".format(prob[0][i]))
            print("Predicted:{}\n".format(c[int(predicted[0][i])]))
        return outputs

c:\Users\Reinaldo yang\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Reinaldo yang\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\Reinaldo yang/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [00:27<00:00, 20.4MB/s] 


##**Visualize the feature maps extracted from the layer-index-5 and layer-index-10**##

In [ ]:
if __name__=='__main__':
  # get class
  c = {}
  with open("imagenet1000_clsidx_to_labels.txt") as f:
    for line in f:
      (key, val) = line.split(":")
      c[int(key)] = val.split(",")[0]
  # Define image path and select the layer
  myClass=FeatureVisualization('./dog.jpg',5)
  print(myClass.pretrained_model2)

  myClass.save_feature_to_img()
  myClass.predict()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

##**Example 6~8**##

In [7]:
if __name__=='__main__':
    # get class
    c = {}
    with open("imagenet1000_clsidx_to_labels.txt") as f:
        for line in f:
            (key, val) = line.split(":")
            c[int(key)] = val.split(",")[0]
    # Define image path and select the layer
    myClass=FeatureVisualization('./dog.jpg',5)
    Compare=FeatureVisualization('./dog.jpg',5)    
    print(myClass.pretrained_model2)

    myClass.save_feature_to_img()
    Compare.save_feature_to_img1()
    print("The first picture classification predict:")
    myClass_vector = myClass.predict()
    print("The second picture classification predict:")
    Compare_vector = Compare.predict()
    #Define cosine similarity
    cos= nn.CosineSimilarity(dim=1)
    #Define Euclidean distance
    euclidean_dist = torch.dist(myClass_vector,Compare_vector,p=2)
    cosine_dist = cos(myClass_vector,Compare_vector)
    print("Verification:")
    if cosine_dist < 0.6:
        print("They are the same!")
        print("Their cosine_distance:{}".format(cosine_dist))
    else:
        print("They are not the same!")
        print("Their cosine_distance:{}".format(cosine_dist))
       
    print("Their euclidean_dist:{}".format(euclidean_dist))

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

##**Example 9~13**##
##**Use the VGG-16 model pretrained on ImageNet to train the CIFAR-100 dataset**##

In [9]:
transform = transforms.Compose([
                 transforms.Resize(32), 
                 transforms.ToTensor(), 
                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

batch_size = 256
learning_rate = 0.001
input_size = 32
num_epoch = 1
 
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=0)
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=0)

 17%|█▋        | 28999680/169001437 [04:22<21:05, 110661.15it/s] 


KeyboardInterrupt: 

In [ ]:
my_model = torchvision.models.vgg16_bn(pretrained=True)
# print(my_model)
# Parameters of newly constructed modules have requires_grad=True by default

# my_model.features = torch.nn.Sequential(
#     nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
#     nn.ReLU(inplace=True),
#     nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
#     nn.ReLU(inplace=True),
#     nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
#     nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
#     nn.ReLU(inplace=True),
#     nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
#     nn.ReLU(inplace=True),
#     nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
#     nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
#     nn.ReLU(inplace=True),
#     nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
#     nn.ReLU(inplace=True),
#     nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
#     nn.ReLU(inplace=True),
#     nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
#     nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
#     nn.ReLU(inplace=True),
#     nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
#     nn.ReLU(inplace=True),
#     nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
#     nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
#     nn.ReLU(inplace=True),
#     nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
#     nn.ReLU(inplace=True),
#     nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
# )
# my_model.avgpool = torch.nn.AdaptiveAvgPool2d(output_size=(7, 7))
my_model.classifier = torch.nn.Sequential(torch.nn.Linear(25088, 4096),
                                       torch.nn.ReLU(),
                                       torch.nn.Dropout(p=0.5),
                                       torch.nn.Linear(4096, 1024),
                                       torch.nn.ReLU(),
                                       torch.nn.Dropout(p=0.5),
                                       torch.nn.Linear(1024, 100))
print(my_model)

my_model = my_model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
# optimizer = optim.SGD(my_model.classifier.parameters(), lr=learning_rate, momentum=0.9)
optimizer = torch.optim.Adam(my_model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
# print(my_model)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [ ]:
for epoch in range(num_epoch):  # loop over the dataset multiple times
    myClass=FeatureVisualization('./dog.jpg',7, models=my_model)
    myClass.save_feature_to_img(epoch=epoch)
    running_loss = 0.0
    my_model.train()
    for i, data in enumerate(trainloader):
        # zero the parameter gradients
        optimizer.zero_grad()

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        inputs = Variable(inputs.cuda())
        labels = Variable(labels.cuda())
      
        # forward + backward + optimize
        outputs = my_model(inputs)



        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        # if i % 2000 == 1999:    # print every 2000 mini-batches
        print('epochs:　%d, steps: %d loss: %.3f' % (epoch + 1, i + 1, running_loss))
        running_loss = 0.0

print('Finished Training')

correct = 0
total = 0
with torch.no_grad():
    my_model.eval()
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = my_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Accuracy of the network on the test images: %d %%' % accuracy)

torch.Size([1, 128, 112, 112])
On layer:7, We can get the 128 feature maps
epochs:　1, steps: 1 loss: 4.604
epochs:　1, steps: 2 loss: 4.904
epochs:　1, steps: 3 loss: 4.537
epochs:　1, steps: 4 loss: 4.638
epochs:　1, steps: 5 loss: 4.641
epochs:　1, steps: 6 loss: 4.646
epochs:　1, steps: 7 loss: 4.574
epochs:　1, steps: 8 loss: 4.557
epochs:　1, steps: 9 loss: 4.592
epochs:　1, steps: 10 loss: 4.540
epochs:　1, steps: 11 loss: 4.498
epochs:　1, steps: 12 loss: 4.559
epochs:　1, steps: 13 loss: 4.399
epochs:　1, steps: 14 loss: 4.539
epochs:　1, steps: 15 loss: 4.603
epochs:　1, steps: 16 loss: 4.555
epochs:　1, steps: 17 loss: 4.411
epochs:　1, steps: 18 loss: 4.523
epochs:　1, steps: 19 loss: 4.474
epochs:　1, steps: 20 loss: 4.490
epochs:　1, steps: 21 loss: 4.493
epochs:　1, steps: 22 loss: 4.510
epochs:　1, steps: 23 loss: 4.554
epochs:　1, steps: 24 loss: 4.488
epochs:　1, steps: 25 loss: 4.523
epochs:　1, steps: 26 loss: 4.480
epochs:　1, steps: 27 loss: 4.592
epochs:　1, steps: 28 loss: 4.457
epochs:　1,

In [ ]:
def preprocess_image(cv2im, resize_im=True):

    # Resize image
    if resize_im:
        cv2im = cv2.resize(cv2im, (224, 224))
    im_as_arr = np.float32(cv2im)
    im_as_arr = np.ascontiguousarray(im_as_arr[..., ::-1])
    im_as_arr = im_as_arr.transpose(2, 0, 1)  # Convert array to D,W,H
    # Normalize the channels
    for channel, _ in enumerate(im_as_arr):
        im_as_arr[channel] /= 255
    # Convert to float tensor
    im_as_ten = torch.from_numpy(im_as_arr).float()
    # Add one more channel to the beginning. Tensor shape = 1,3,224,224
    im_as_ten.unsqueeze_(0)
    # Convert to Pytorch variable
    im_as_var = Variable(im_as_ten, requires_grad=True)
    return im_as_var

In [ ]:
class Pretrained_VGGNet():
    def __init__(self,img_path):
        self.img_path=img_path
        # Load pretrained model
        self.pretrained_model = my_model
        self.pretrained_model = self.pretrained_model.to(device)
        self.pretrained_model.eval()
    def process_image(self):
        img=cv2.imread(self.img_path)
        img=preprocess_image(img)
        return img

    def get_single_feature(self):
        # Get the feature map
        features=self.get_feature()
        feature=features[:,0,:,:]
        feature=feature.view(feature.shape[1],feature.shape[2])

        #print(features)
        #print(feature.shape)
        return feature

    def plot_probability(self,outputs):

        outputs = outputs.data.cpu().numpy()
        outputs = np.ndarray.tolist(outputs)

        x = range(0, 100)        
        plt.bar(x, outputs[0])
        plt.xlabel("Class")
        plt.ylabel("Probability")
        plt.title("Image classifier")
        plt.show()

    def predict(self):
        input=self.process_image()
        input = input.to(device).cuda()
        outputs = self.pretrained_model(input)

        s = torch.nn.Softmax(dim=1)
        result = s(outputs)
        # self.plot_probability(result)

        prob, predicted = result.sort(1,descending=True)
        prob = prob.data.cpu().numpy()

        predicted = predicted.data.cpu().numpy()
        
        print("Probability TOP-3:\n")
        print("")
        for i in range(3):
            
            print("TOP_"+str(i+1))
            print("Probability:{}".format(prob[0][i]))
            print("Predicted:{}\n".format(c[int(predicted[0][i])]))
        return outputs

In [ ]:
if __name__=='__main__':
  # get class
  c = {}
  with open("./CIFAR100_clsidx_to_labels.txt") as f:
    for line in f:
      (key, val) = line.split(":")
      c[int(key)] = val.split(",")[0]
  # Define image path
  myClass=Pretrained_VGGNet('./dog.jpg')
  print(myClass.pretrained_model)
  myClass.predict()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

##**Example 14~18**##
##**Use the VGG-16 model train the CIFAR-100 dataset from scratch**##